In [1]:
import numpy as np
import arviz as az
import pymc3 as pm
import theano

In [2]:
pm.__version__, theano.__version__

('3.11.0', '1.1.2')

## Truncated Exponential


In [ ]:
# create data
np.random.seed(451)
x = np.random.exponential(3, size=500)
minx=1
maxx=20

obs = x[np.where(~((x<minx) | (x>maxx)))] # remove values outside range
print(obs.size)

In [ ]:
with pm.Model() as m:
    mu = pm.Exponential("mu", lam=1/5)
    x = pm.Exponential('x', lam=1/mu, observed=obs)

    exp_dist = pm.Exponential.dist(lam=1/mu) # this is not part of the model, just used to get the logcdf
    norm_term = pm.Potential("norm_term", -pm.math.logdiffexp(exp_dist.logcdf(maxx), exp_dist.logcdf(minx)) * x.size)

    trace = pm.sample(return_inferencedata=True)
#
print(az.summary(trace))

In [ ]:
print(m.logp({'mu_log__': np.log(3)}))
print(m.dlogp_array(np.log(3)))


In [ ]:
with pm.Model() as m:
    mu = pm.Exponential("mu", lam=1/5)
    x = pm.Truncated(pm.Exponential, lower=minx, upper=maxx)(
        'x', lam=1/mu, observed=obs, shape=len(obs)
    )
    trace = pm.sample(return_inferencedata=True, )#step=[pm.Metropolis([mu])])

print(az.summary(trace))

In [ ]:
print(m.logp({'mu_log__': np.log(3)}))
print(m.dlogp_array(np.log(3)))

In [ ]:
with pm.Model() as m:
    mu = pm.Exponential("mu", lam=1/5)  # prior exponential with mean of 5
    x = pm.Bound(pm.Exponential, lower=minx, upper=maxx)(
        'x', lam=1/mu, observed=obs
    ) # obs exponential with mean of $\lambda$.
    trace = pm.sample(return_inferencedata=True)

print(az.summary(trace))

In [ ]:
print(m.logp({'mu_log__': np.log(3)}))
print(m.dlogp_array(np.log(3)))


# TruncatedNormal

In [3]:
np.random.seed(2021)
x = np.random.normal(0, 2, size=5000)
obs = x[(x >= -1) & (x <= 2)]

In [4]:
with pm.Model() as m1:
    mu = pm.Normal('mu', 0, 5)
    x = pm.Normal('x', mu=mu, sigma=2, observed=obs)
#     trace = pm.sample(return_inferencedata=True)
#
# az.summary(trace)

In [ ]:
print(m1.logp({'mu': 0}))
print(m1.dlogp([mu])({'mu':0}))
print(m1.dlogp_array(0))

In [5]:
with pm.Model() as m2:
    mu = pm.Normal('mu', 0, 5)
    x = pm.TruncatedNormal('x', mu=mu, sigma=2, lower=-1, upper=2, observed=obs)
#     trace = pm.sample(return_inferencedata=True)
#
# az.summary(trace)

In [6]:
print(m2.logp({'mu': 0}))
print(m2.dlogp([mu])({'mu':0}))
print(m2.dlogp_array(0))

WARNING (theano.tensor.opt): Optimization Warning: The Op erfcx does not provide a C implementation. As well as being potentially slow, this also disables loop fusion.


-2884.7483443333817
[12.84180893]
[12.84180893]


In [ ]:
# theano.printing.debugprint(pm.theanof.gradient(m2.logpt, [mu]))
# theano.printing.debugprint(m2.mu.logpt)

In [ ]:
with pm.Model() as m3:
    mu = pm.Normal('mu', 0, 5)
    x = pm.Truncated(pm.Normal, lower=-1, upper=2)('x', mu=mu, sigma=2, observed=obs)
#     trace = pm.sample(return_inferencedata=True)
#
# az.summary(trace)

In [ ]:
print(m3.logp({'mu': 0}))
print(m3.dlogp([mu])({'mu':0}))
print(m3.dlogp_array(0))

In [ ]:
with pm.Model() as m4:
    mu = pm.Normal('mu', 0, 5)
    x = pm.Bound(pm.Normal, lower=-1, upper=2)('x', mu=mu, sigma=2, observed=obs)
    # trace = pm.sample(return_inferencedata=True, step=[pm.Metropolis([mu])])
#
# print(az.summary(trace))

In [ ]:
print(m4.logp({'mu': 0}))
print(m4.dlogp_array(0))

In [ ]:
with pm.Model() as m5:
    mu = pm.Normal("mu", 0, 5)
    x = pm.Normal('x', mu=mu, sigma=2, observed=obs)

    norm_dist = pm.Normal.dist(mu=mu, sigma=2) # this is not part of the model, just used to get the logcdf
    norm_term = pm.Potential("norm_term", -pm.math.logdiffexp(norm_dist.logcdf(2), norm_dist.logcdf(-1)) * x.size)

#     trace = pm.sample(return_inferencedata=True)
#
# print(az.summary(trace))

In [ ]:
print(m5.logp({'mu': 0}))
print(m5.dlogp([mu])({'mu': 0}))
print(m5.dlogp_array(0))
